# `CrystalMap` API

* Quick and dirty description of the (more or less) full `CrystalMap` API so far.
* .ang and .h5ebsd files for testing are temporarily available here: http://folk.ntnu.no/hakonwii/files/orix-demos/

<!--
# Introduction

This notebook illustrates clustering of Ti crystal orientations using data obtained from a highly deformed specimen, using EBSD.

This functionaility has been checked to run in orix-0.2.0 (December 2019). Bugs are always possible, do not trust the code blindly, and if you experience any issues please report them here: https://github.com/pyxem/orix-demos/issues
-->

# Contents

1. <a href='#load'> Load CrystalMap objects from file</a>
2. <a href='#init'> Initialize a CrystalMap object</a>
3. <a href='#crystalmap'> CrystalMap</a>
4. <a href='#phaselist'> PhaseList</a>
5. <a href='#phase'> Phase</a>
6. <a href='#examples'> Examples</a>
7. <a href="#gotchas"> Gotchas</a>

Import orix classes and various dependencies

In [1]:
%matplotlib qt5

# Important external dependencies
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import h5py  # Needed for h5ebsd reader

# orix dependencies (not tested)
from orix.crystal_map import CrystalMap, PhaseList, Phase
from orix.quaternion.rotation import Rotation
from orix import plot
from orix.io import load_ang, load_emsoft

Set input file and file path

In [2]:
datadir = '/home/hakon/phd/data/jarle_emsoft/sdss/em'
emsoft_file = os.path.join(datadir, 'sdss_austenite_dp.h5')
ang_file = os.path.join(datadir, 'sdss_ferrite_austenite.ang')

# <a id='load'></a> 1. Load CrystalMap objects from file

The .h5 file comes from dictionary indexing in EMsoft. A map of super-duplex stainless steel of size (100, 117) has been indexed with both austenite and ferrite. This file contains the results from the austenite indexing. Let's load it with `orix.io.load_emsoft`.

In [3]:
cm2 = load_emsoft(emsoft_file)

print(cm2)

Phase    Orientations       Name  Symmetry     Color
    0  11700 (100.0%)  austenite      m-3m  tab:blue
Properties: AvDotProductMap, CI, CIMap, IQ, IQMap, ISM, ISMap, KAM, OSM, RefinedDotProducts, TopDotProductList, TopMatchIndices
Scan unit: px


The .ang file used here comes from EMsoft's `EMdpmerge`, where the austenite and ferrite results have been merged. Note that the only property maps available are the image quality and dot product maps.

The ang file reader `orix.io.load_ang` should recognize the ang file formats from EDAX TSL (v7), ASTAR Index, and EMsoft.

In [4]:
cm = load_ang(ang_file)

print(cm)

Phase   Orientations       Name  Symmetry       Color
    1   5657 (48.4%)  austenite       432    tab:blue
    2   6043 (51.6%)    ferrite       432  tab:orange
Properties: iq, dp
Scan unit: px


# <a id='init'></a> 2. Initialize a CrystalMap object

A `CrystalMap` object can also be initialized manually. Let's do this for the .ang file

In [5]:
euler1, euler2, euler3, x, y, iq, dp, phase = np.loadtxt(
    ang_file,
    usecols=(0, 1, 2, 3, 4, 5, 6, 7),
    unpack=True
)

euler = np.column_stack((euler1, euler2, euler3))

Set up properties

In [6]:
properties = {"dp": dp, "iq": iq}

Inspect the crystal map initialization by printing its `__init__` docstring

In [7]:
print(CrystalMap.__init__.__doc__)    


        Parameters
        ----------
        rotations : orix.quaternion.rotation.Rotation
            Rotation of each data point. Must contain only one spatial
            dimension in the first array axis. May contain multiple
            rotations per point, included in the second array axes. Crystal
            map data size is set equal to the first array axis' size.
        phase_id : numpy.ndarray, optional
            Phase ID of each pixel. IDs equal to -1 are considered not
            indexed. If ``None`` is passed (default), all points are
            considered to belong to one phase with ID 1.
        x : numpy.ndarray, optional
            Map x coordinate of each data point. If ``None`` is passed,
            the map is assumed to be 1D, and it is set to an array of
            increasing integers from 0 to the length of the `phase_id`
            array.
        y : numpy.ndarray, optional
            Map y coordinate of each data point. If ``None`` is passed,
      

Initialize the object, also setting the scan unit

In [8]:
cm = CrystalMap(
    rotations=Rotation.from_euler(euler),
    phase_id=phase,
    x=x,
    y=y,
    phase_name=["austenite", "ferrite"],
    symmetry=["432", "432"],
    prop=properties,
)
cm.scan_unit = "um"

print(cm)

Phase   Orientations       Name  Symmetry       Color
    1   5657 (48.4%)  austenite       432    tab:blue
    2   6043 (51.6%)    ferrite       432  tab:orange
Properties: dp, iq
Scan unit: um


# <a id='crystalmap'></a> 3. CrystalMap

Print class description

In [9]:
print(CrystalMap.__doc__)

Crystallographic map of rotations, crystal phases and key properties
    associated with every spatial coordinate in a 1D, 2D or 3D space.

    All properties are stored as 1D arrays, and reshaped when necessary.

    Attributes
    ----------
    all_indexed : bool
        Whether all data points are indexed.
    dx, dy, dz : float
        Step sizes in x, y and z directions.
    id : int
        Data point ID.
    is_indexed : numpy.ndarray
        Boolean array with True for indexed data points.
    ndim : int
        Number of map dimensions.
    orientations : orix.quaternion.orientation.Orientation
        Orientation(s) of each data point.
    phase_id : numpy.ndarray
        Phase ID of each data point.
    phases : orix.crystal_map.PhaseList
        List of phases with their IDs, names, crystal symmetries and
        colors (possibly more than are in the data).
    phases_in_data : orix.crystal_map.PhaseList
        List of phases in the data, with their IDs, names, crystal
  

`__repr__` (inspired by MTEX)

In [10]:
cm

Phase   Orientations       Name  Symmetry       Color
    1   5657 (48.4%)  austenite       432    tab:blue
    2   6043 (51.6%)    ferrite       432  tab:orange
Properties: dp, iq
Scan unit: um

Custom, private attributes

In [11]:
[i for i in dir(cm) if i.startswith('_') and not i.endswith('__')]

['_coordinates',
 '_data_shape_from_coordinates',
 '_data_slices_from_coordinates',
 '_id',
 '_original_shape',
 '_phase_id',
 '_prop',
 '_rotations',
 '_step_size_from_coordinates',
 '_step_sizes',
 '_x',
 '_y',
 '_z']

Public attributes and methods

In [12]:
[i for i in dir(cm) if not i.startswith('_')]

['all_indexed',
 'deepcopy',
 'dx',
 'dy',
 'dz',
 'get_map_data',
 'id',
 'is_in_data',
 'is_indexed',
 'ndim',
 'orientations',
 'phase_id',
 'phases',
 'phases_in_data',
 'prop',
 'rotations',
 'rotations_per_point',
 'scan_unit',
 'shape',
 'size',
 'x',
 'y',
 'z']

Docstrings of public attributes and their values in the current CrystalMap instance

In [13]:
print(CrystalMap.all_indexed.__doc__)
cm.all_indexed

Return whether all points in data are indexed.


True

In [14]:
print(CrystalMap.is_indexed.__doc__)
cm.is_indexed

Return whether points in data are indexed.


array([ True,  True,  True, ...,  True,  True,  True])

In [15]:
print(CrystalMap.ndim.__doc__)
cm.ndim

Return number of data dimensions of points in the data.


2

In [16]:
cma = cm["austenite"]

In [17]:
cma.rotations

Rotation (5657,)
[[ 0.8686 -0.3569  0.2749  0.2064]
 [ 0.8681 -0.3581  0.2744  0.2068]
 [ 0.8684 -0.3578  0.2751  0.2052]
 ...
 [ 0.9639 -0.022  -0.0754  0.2545]
 [ 0.8854 -0.3337  0.2385 -0.2187]
 [ 0.885  -0.3341  0.2391 -0.2193]]

In [18]:
print(CrystalMap.orientations.__doc__)
cm['austenite'].orientations

Return an Orientation object of orientations in data.


Orientation (5657,) 1, 432
[[ 0.8686 -0.3569  0.2749  0.2064]
 [ 0.8681 -0.3581  0.2744  0.2068]
 [ 0.8684 -0.3578  0.2751  0.2052]
 ...
 [ 0.9639 -0.022  -0.0754  0.2545]
 [ 0.8854 -0.3337  0.2385 -0.2187]
 [ 0.885  -0.3341  0.2391 -0.2193]]

In [19]:
print(CrystalMap.phase_id.__doc__)
cm.phase_id

Return phase IDs of points in the data.


array([2, 1, 1, ..., 2, 1, 1])

In [20]:
# Set during __init__, not a derived attribute, hence no docstring available
cm.phases  # similar CrystalMap's __repr__

Id       Name  Symmetry       Color
 1  austenite       432    tab:blue
 2    ferrite       432  tab:orange

In [21]:
print(CrystalMap.phases_in_data.__doc__)
cm.phases_in_data

Return a list of phases in the data.

        This is needed because it can be useful to have phases not in the
        data but in `self.phases`.
        


Id       Name  Symmetry       Color
 1  austenite       432    tab:blue
 2    ferrite       432  tab:orange

In [22]:
print(CrystalMap.prop.__doc__)
cm.prop

Return a :class:`~orix.crystal_map.CrystalMapProperties`
        dictionary with data properties.
        


{'dp': array([0.799, 0.797, 0.825, ..., 0.817, 0.809, 0.828]),
 'iq': array([24.4, 24. , 30.3, ..., 25.3, 25.8, 31.7])}

In [23]:
print(CrystalMap.rotations.__doc__)
cm.rotations

Return a Rotation object of rotations in the data.


Rotation (11700,)
[[ 0.9358 -0.3191 -0.0919  0.1185]
 [ 0.8686 -0.3569  0.2749  0.2064]
 [ 0.8681 -0.3581  0.2744  0.2068]
 ...
 [ 0.9126  0.3022  0.1552 -0.2275]
 [ 0.8854 -0.3337  0.2385 -0.2187]
 [ 0.885  -0.3341  0.2391 -0.2193]]

In [24]:
print(CrystalMap.shape.__doc__)
cm.shape

Return shape of points in the data.


(100, 117)

In [25]:
print(CrystalMap.size.__doc__)
print(cm.size)
print(cm["austenite"].size)

Return total number of points in the data.
11700
5657


Indexing/slicing ...

... by map position (slices)

In [26]:
(y0, y1) = (20, 40)
(x0, x1) = (50, 60)
cm1 = cm[y0:y1, x0:x1]
cm1

Phase   Orientations       Name  Symmetry       Color
    1    148 (74.0%)  austenite       432    tab:blue
    2     52 (26.0%)    ferrite       432  tab:orange
Properties: dp, iq
Scan unit: um

In [27]:
cm1.size

200

... by phase name

In [28]:
cm2 = cm['austenite']
cm2

Phase   Orientations       Name  Symmetry     Color
    1  5657 (100.0%)  austenite       432  tab:blue
Properties: dp, iq
Scan unit: um

In [29]:
cm2.size / cm.size

0.4835042735042735

In [30]:
cm['austenite', 'ferrite']

Phase   Orientations       Name  Symmetry       Color
    1   5657 (48.4%)  austenite       432    tab:blue
    2   6043 (51.6%)    ferrite       432  tab:orange
Properties: dp, iq
Scan unit: um

... by (chained) conditional(s)

In [31]:
cm[cm.phase_id == 1]

Phase   Orientations       Name  Symmetry     Color
    1  5657 (100.0%)  austenite       432  tab:blue
Properties: dp, iq
Scan unit: um

In [32]:
cm[cm.dp > 0.81]

Phase   Orientations       Name  Symmetry       Color
    1   4092 (44.8%)  austenite       432    tab:blue
    2   5035 (55.2%)    ferrite       432  tab:orange
Properties: dp, iq
Scan unit: um

In [33]:
cm[(cm.iq > np.mean(cm.iq)) & (cm.phase_id == 1)]

Phase   Orientations       Name  Symmetry     Color
    1  1890 (100.0%)  austenite       432  tab:blue
Properties: dp, iq
Scan unit: um

Add property

In [34]:
cm.prop['dp_times_iq'] = cm.dp * cm.iq
print(cm)
print("\n", cm.dp_times_iq)

Phase   Orientations       Name  Symmetry       Color
    1   5657 (48.4%)  austenite       432    tab:blue
    2   6043 (51.6%)    ferrite       432  tab:orange
Properties: dp, iq, dp_times_iq
Scan unit: um

 [19.4956 19.128  24.9975 ... 20.6701 20.8722 26.2476]


# <a id='phaselist'></a> 4. PhaseList

In [35]:
print(PhaseList.__doc__)

A list of phases in a crystallographic map.

    Each phase in the list must have a unique phase id and name.

    Attributes
    ----------
    size : int
        Number of phases in list.
    names : list of str
        List of phase names.
    symmetries : list of orix.quaternion.symmetry.Symmetry
        List of phase crystal symmetries.
    colors : list of tuple
        List of tuples with three entries, RGB, defining phase colors.
    phase_ids : list of int
        List of unique phase indices in a crystallographic map as imported.

    Methods
    -------
    deepcopy()
        Return a deep copy using :py:func:`~copy.deepcopy` function.
    add_not_indexed()
        Add a dummy phase to assign to not indexed data points.
    sort_by_id()
        Sort list according to phase ID.
    


Get from CrystalMap

In [36]:
phases = cm.phases  # As shown above
phases

Id       Name  Symmetry       Color
 1  austenite       432    tab:blue
 2    ferrite       432  tab:orange

Custom, private attributes

In [37]:
[i for i in dir(phases) if i.startswith('_') and not i.endswith('__')]

['_dict']

Public attributes

In [38]:
[i for i in dir(phases) if not i.startswith('_')]

['add_not_indexed',
 'colors',
 'colors_rgb',
 'deepcopy',
 'names',
 'phase_ids',
 'size',
 'sort_by_id',
 'symmetries']

Inspect properties

In [39]:
print(PhaseList.size.__doc__)
phases.size

Return number of phases in the list.


2

In [40]:
print(PhaseList.phase_ids.__doc__)
phases.phase_ids

Return unique phase IDs in the list of phases.


[1, 2]

In [41]:
print(PhaseList.names.__doc__)
phases.names

Return a list of phase names in the list.


['austenite', 'ferrite']

In [42]:
print(PhaseList.symmetries.__doc__)
phases.symmetries

Return a list of crystal symmetries of phases in the list.


[Symmetry (24,) 432
 [[ 1.      0.      0.      0.    ]
  [ 0.7071  0.      0.      0.7071]
  [ 0.      0.      0.      1.    ]
  [-0.7071  0.      0.      0.7071]
  [ 0.5     0.5     0.5     0.5   ]
  [ 0.      0.      0.7071  0.7071]
  [-0.5    -0.5     0.5     0.5   ]
  [-0.7071 -0.7071  0.      0.    ]
  [ 0.      1.      0.      0.    ]
  [ 0.      0.7071  0.7071  0.    ]
  [ 0.      0.      1.      0.    ]
  [ 0.     -0.7071  0.7071  0.    ]
  [-0.5     0.5     0.5    -0.5   ]
  [ 0.      0.      0.7071 -0.7071]
  [ 0.5    -0.5     0.5    -0.5   ]
  [ 0.7071 -0.7071  0.      0.    ]
  [ 0.      0.7071  0.      0.7071]
  [-0.5     0.5     0.5     0.5   ]
  [-0.7071  0.      0.7071  0.    ]
  [-0.5    -0.5     0.5    -0.5   ]
  [ 0.      0.7071  0.     -0.7071]
  [ 0.5     0.5     0.5    -0.5   ]
  [ 0.7071  0.      0.7071  0.    ]
  [ 0.5    -0.5     0.5     0.5   ]],
 Symmetry (24,) 432
 [[ 1.      0.      0.      0.    ]
  [ 0.7071  0.      0.      0.7071]
  [ 0.      0.      0.

In [43]:
print(PhaseList.colors.__doc__)
phases.colors

Return a list of phase color names in the list.


['tab:blue', 'tab:orange']

In [44]:
print(PhaseList.colors_rgb.__doc__)
phases.colors_rgb

Return a list of phase color RGB values in the list.


[(0.12156862745098039, 0.4666666666666667, 0.7058823529411765),
 (1.0, 0.4980392156862745, 0.054901960784313725)]

Add "not_indexed" to phase list

In [45]:
print(PhaseList.add_not_indexed.__doc__)
phases.add_not_indexed()
print(phases)

Add a dummy phase to assign to not indexed data points.

        The phase, named "not_indexed", has a "symmetry" equal to None, and
        a white color when plotted.
        
Id         Name  Symmetry       Color
-1  not_indexed      None           w
 1    austenite       432    tab:blue
 2      ferrite       432  tab:orange



Initialize a PhaseList...

In [46]:
print(PhaseList.__init__.__doc__)


        Parameters
        ----------
        phases : orix.crystal_map.Phase, a list of orix.crystal_map.Phase                or a dictionary of orix.crystal_map.Phase
            A list or dict of phases or a single phase. The other arguments
            are ignored if this is passed.
        names : str or list of str
            Phase names.
        symmetries : str or list of str
            Point group symmetries.
        colors : str or list of str
            Phase colors.
        phase_ids : int, list of int or numpy.ndarray of int
            Phase IDs.
        


... from input

In [47]:
PhaseList(
    names=['al', 'cu'],
    symmetries=['m-3m', 'm3m'],  # Note that m3m = m-3m
    colors=['lime', 'xkcd:violet'],
    phase_ids=[0, 1],
)

Id  Name  Symmetry        Color
 0    al      m-3m         lime
 1    cu      m-3m  xkcd:violet

... from a list of Phase objects

In [48]:
al = Phase(name='al', symmetry='m-3m')
cu = Phase(name='cu')

#PhaseList([al, cu])  # Not working, will implement
PhaseList(al)

Id  Name  Symmetry     Color
 0    al      m-3m  tab:blue

Index PhaseList...

In [49]:
print(PhaseList.__getitem__.__doc__)
# This doc should perhaps be moved to a more accessible place

Return a PhaseList or a Phase object, depending on input.

        Examples
        --------
        A PhaseList object can be indexed in multiple ways.

        >>> pl = PhaseList(names=['a', 'b'], symmetries=['1', '3'])
        >>> pl
        Id  Name  Symmetry  Color
        0   a     1         tab:blue
        1   b     3         tab:orange

        Return a Phase object

        >>> pl[0]  # Index with a single phase id
        <name: a. symmetry: 1. color: tab:blue>
        >>> pl['b']  # Index with a phase name
        <name: b. symmetry: 3. color: tab:orange>

        Return a PhaseList object

        >>> pl[0:]  # Index with slices
        Id  Name  Symmetry  Color
        0   a     1         tab:blue
        1   b     3         tab:orange
        >>> pl['a', 'b']  # Index with a tuple of phase names
        Id  Name  Symmetry  Color
        0   a     1         tab:blue
        1   b     3         tab:orange
        >>> pl[0, 1]  # Index with a tuple of phase ids
        Id  

... by phase name

In [50]:
phases['austenite']

<name: austenite. symmetry: 432. color: tab:blue>

In [51]:
phases['austenite', 'ferrite']

Id       Name  Symmetry       Color
 1  austenite       432    tab:blue
 2    ferrite       432  tab:orange

... by phase id

In [52]:
phases[1]

<name: austenite. symmetry: 432. color: tab:blue>

In [53]:
phases[1, 2]

Id       Name  Symmetry       Color
 1  austenite       432    tab:blue
 2    ferrite       432  tab:orange

... by slice

In [54]:
phases[1:]

Id       Name  Symmetry       Color
 1  austenite       432    tab:blue
 2    ferrite       432  tab:orange

Add phase to PhaseList

In [55]:
phases2 = phases.deepcopy()
phases2['sigma'] = '4/mmm'
phases2

Id         Name  Symmetry       Color
-1  not_indexed      None           w
 1    austenite       432    tab:blue
 2      ferrite       432  tab:orange
 3        sigma     4/mmm   tab:green

# <a id='phase'></a> 5. Phase

In [56]:
print(Phase.__doc__)

Name, crystal symmetry, and color of a phase in a crystallographic
    map.

    Attributes
    ----------
    name : str
        Phase name.
    symmetry : orix.quaternion.symmetries.Symmetry
        Crystal symmetries of the phase.
    color : str
        Name of phase color in Matplotlib's list of named colors.
    color_rgb : tuple
        RGB values of phase color, obtained from the color name.

    Methods
    -------
    deepcopy()
        Return a deep copy using :py:func:`~copy.deepcopy` function.
    


No custom, private attributes.

Public attributes

In [57]:
[i for i in dir(Phase) if not i.startswith('_')]

['color', 'color_rgb', 'deepcopy', 'name', 'symmetry']

In [58]:
print(Phase.color.__doc__)

Return phase color.


From PhaseList by...

... phase name

In [59]:
phases['austenite']

<name: austenite. symmetry: 432. color: tab:blue>

... phase ID

In [60]:
phases[1]

<name: austenite. symmetry: 432. color: tab:blue>

Initialize

In [61]:
Phase('au', symmetry='m-3m', color='salmon')

<name: au. symmetry: m-3m. color: salmon>

# <a id='examples'></a> 6. Examples

All map plotting is done via a so-called Matplotlib "projection" named "plot_map". To plot a phase map

In [62]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm)

Hover over figure pixels to display the (x,y) position and orientations in that pixel!

We can set new colors if we aren't satisfied with Matplotlib's default colors

In [63]:
cm.phases["austenite"].color = (0, 1, 1)
cm.phases["ferrite"].color = "xkcd:violet"

fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm)

We can add any overlay to our map

In [64]:
ax.add_overlay(cm, cm.dp)

Save phase map with scalebar and legend but without white padding

In [65]:
ax.remove_padding()
fig.savefig(
    os.path.join(datadir, 'phase_map.png'),
    bbox_inches="tight",
    pad_inches=0,
    dpi=300,  # Dots per inch
)

Save phase map without scalebar, legend and white padding, and one image pixel per map pixel 

In [66]:
plt.imsave(
    os.path.join(datadir, 'phase_no_fluff.png'),
    arr=im.get_array()
)

We can plot arbitrary properties

In [67]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, cm.dp, cmap="inferno")

And change the colormap later

In [68]:
im.set_cmap("viridis")

And add a colorbar if we want

In [69]:
cbar = ax.add_colorbar(title="Oops")

In [70]:
cbar.ax.set_ylabel("Dot product", rotation=270);

We can also plot orientation related values, like axis and angles etc., and restrict the color bar maximum

In [71]:
# Get rotation angles in degrees
angles = cm.rotations.angle.data * 180 / np.pi

fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, angles, vmax=angles.max() - 10)
ax.add_overlay(cm, cm.dp)
ax.add_colorbar(title="Rotation angle");

In [72]:
cm.phases

Id         Name  Symmetry        Color
-1  not_indexed      None            w
 1    austenite       432         aqua
 2      ferrite       432  xkcd:violet

In [73]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, "orientations")
ax.add_overlay(cm, cm.dp)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


In [74]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, cm.rotations.axis)
ax.add_overlay(cm, cm.dp)

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Plot only one phase, while passing custom:
* scalebar properties (https://matplotlib.org/mpl_toolkits/axes_grid/api/anchored_artists_api.html#mpl_toolkits.axes_grid1.anchored_artists.AnchoredSizeBar)
* legend properties (https://matplotlib.org/3.2.1/api/_as_gen/matplotlib.pyplot.legend.html)

In [75]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(
    cm["austenite"],
    scalebar=True,  # False for removed
    scalebar_properties={
        "loc": 4,  # 1: upper right, 2: upper left, etc. counter-clockwise
        "frameon": False,
        "sep": 6,  # Vertical spacing between bar and text
        "size_vertical": 0.2,  # Bar height
    },
    legend_properties={
        "framealpha": 1,  # 0: fully transparent, 1: opaque
        "handlelength": 1.5,  # Colored square width
        "handletextpad": 0.1,  # Horizontal space between square and text
        "borderpad": 0.1,
    },
)

Plot only a square of the map

In [76]:
cm2 = cm[20:50, 50:90]

#fig = plt.figure()
#ax = fig.add_subplot(projection="plot_map")
#ax.plot_map(cm2)
#ax.add_overlay(cm2, cm2.dp)

Plot only parts of a map based on chained conditionals, like belonging to one phase or having a property value above a threshold

In [77]:
# Conditional slicing
cm2 = cm[cm.dp > 0.81]
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
ax.plot_map(cm2, cm2.iq, cmap="gray")
ax.add_colorbar("Image quality")

# Chained conditional slicing
cm2 = cm[(cm.dp > 0.81) & (cm.phase_id == 1)]
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
ax.plot_map(cm2, cm2.dp, cmap="cividis")
ax.add_colorbar("Dot product");

Plot histogram of a property per phase

In [78]:
# Change colors again
cm.phases['ferrite'].color = (1, 0, 0)  # or 'r' or 'red'
cm.phases['austenite'].color = (0, 1, 0)  # or 'lime'

# Property of interest
this_prop = 'dp'

# Plot phase map again to see color changes
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
ax.plot_map(cm)
ax.add_overlay(cm, this_prop)  # Can also pass str as overlay
ax.remove_padding()
fig.savefig(os.path.join(datadir, 'test.png'), pad_inches=0, bbox_inches="tight")

# Declare lists for plotting
data = []
labels = []
colors = []

# Get property values, name and color per phase
for _, p in cm.phases_in_data:
    labels.append(p.name)
    colors.append(p.color)

    # Accessing the property dictionary directly
    data.append(cm[p.name].prop[this_prop])
    # or indirectly
    #data.append(cm[p.name].dp)

# Nice bar plot with property histogram per phase
fig, ax = plt.subplots()
ax.hist(
    data,
    bins=20,
    histtype='bar',
    density=True,
    label=labels,
    color=colors
)
ax.set_xlabel(this_prop)
ax.set_ylabel("Frequency")
ax.legend();

Add a new property to the map, modify it, and plot it

In [79]:
# Zero everywhere
cm.prop["grain_boundary"] = 0  # or np.zeros(cm.size), which is equivalent

In [80]:
# Except at low dot product values
cm[cm.dp < 0.81].grain_boundary = 1

In [81]:
fig = plt.figure()
ax = fig.add_subplot(projection="plot_map")
im = ax.plot_map(cm, cm.grain_boundary, cmap="gray")

# <a id='gotchas'></a> 7. Gotchas

Trying to set a map property in the CrystalMap property dictionary which hasn't been declared will silently fail

In [82]:
# When this is forgotten...
# cm["another_grain_boundary"] = 0

# ... this silently fails
cm[cm.dp < 0.82].another_grain_boundary = 1

print(cm.prop.keys())

dict_keys(['dp', 'iq', 'dp_times_iq', 'grain_boundary'])


An error message *could* be raised if a non-existent property was attempted to be set. The drawback of this would be that the user could not add any arbitrary property to a class instance, as e.g. `matplotlib` allows for all their class instances

In [83]:
fig = plt.figure()
plt.close(fig)
fig.another_grain_boundary = "hello"
print(fig.another_grain_boundary)

hello


`numpy` does not allow this.